In [3]:
## Data Exploration of Sanghi '23 x Simple Database
import numpy as np 
import pandas as pd

from astrodb_utils import find_publication, find_source_in_db, load_astrodb, ingest_source
from simple.utils.spectra import * 
from simple.schema import Spectra
from simple.schema import *
from simple.schema import REFERENCE_TABLES


In [3]:
# Check how many sources are already in the database out of the files 
ultracool = pd.read_csv("sanghi23/Ultracool_Fundamental_Properties_Table.csv")
db = load_astrodb("SIMPLE.sqlite", recreatedb=False, reference_tables=REFERENCE_TABLES)

NameError: name 'load_astrodb' is not defined

In [ ]:
ra = ultracool['ra_j2000_formula']
dec = ultracool['dec_j2000_formula']
name = ultracool['name']
sim_name = ultracool['name_simbadable']

results = []
unavail = []

for i in range(len(sim_name)):
    try:
        search = find_source_in_db(
            db, sim_name[i], ra = ra[i], dec = dec[i], ra_col_name= "ra", dec_col_name="dec")
        results.append(search)
    except Exception as e:
        print(f"Could not find with Simbad name: {sim_name[i]}, continuing search with name: {name[i]}")
        try: 
            search = find_source_in_db(db, name[i], ra = ra[i], dec = dec[i], ra_col_name= "ra", dec_col_name="dec")
            results.append(search)
        except Exception as e:
            print(f"Error at index {i}: {e}")
            unavail.append(name[i])
            continue
        
#Alternative Method 
    #query name is sim_name is null query name = name 
    #else query.name = sim_name, cut down time and lines (avoid searching twice)
    #validate with ingest_source


In [ ]:
results

In [ ]:
print((len(results) / len(sim_name)) * 100, "% was avaible in the SIMPLE database.")
print(len(results), "out of", len(sim_name), ".", (len(sim_name)-(len(results))), "missing.")


100.0 % was avaible in the SIMPLE database.
1054 out of 1054 . 0 missing.


In [6]:
import pandas as pd

# List of unavailable targets (Originally)
unavail = ['PSO J000.2794+16.6237',
 'PSO J009.8334+58.5781',
 'WISEA J032301.88+562558.0',
 'PSO J069.4987+19.6346',
 'PSO J080.3940+23.0999',
 '2MASS J05445741+3705039',
 'PSO J139.5493+39.0380',
 'PSO J154.9622+04.8279',
 'PSO J162.0614+13.9756',
 'PSO J210.6211-26.9300',
 'PSO J260.1789+15.3091',
 'PSO J271.8353+46.9326',
 'PSO J277.3873+26.0116',
 'PSO J299.5032+55.0722',
 'PSO J323.4300-07.5353',
 'PSO J345.2949+07.7665',
 'PSO J348.5125+69.6598']

df = pd.read_csv("sanghi23/Ultracool_Fundamental_Properties_Table.csv")

filtered_df = df[df['name'].isin(unavail)]

filtered_df = filtered_df[['name', 'ref_discovery', 'ra_j2000_formula', 'dec_j2000_formula']]

filtered_df.to_csv('unavailable_sources.csv', index=False)

In [7]:
search_l = []
for index, row in filtered_df.iterrows():
    try:
        search = find_source_in_db(db, row['name'], ra=row['ra_j2000_formula'], dec=row['dec_j2000_formula'], ra_col_name="ra", dec_col_name="dec")
        print(f"Source {row['name']} found in the database.")
        search_l.append(search)
        #print(search)
    except Exception as e:
       print(f"Source {row['name']} NOT found in the database. Error: {e}")

print(search_l)

Source PSO J000.2794+16.6237 found in the database.
Source PSO J009.8334+58.5781 found in the database.
Source WISEA J032301.88+562558.0 found in the database.
Source PSO J069.4987+19.6346 found in the database.
Source PSO J080.3940+23.0999 found in the database.
Source 2MASS J05445741+3705039 found in the database.
Source PSO J139.5493+39.0380 found in the database.
Source PSO J154.9622+04.8279 found in the database.
Source PSO J162.0614+13.9756 found in the database.
Source PSO J210.6211-26.9300 found in the database.
Source PSO J260.1789+15.3091 found in the database.
Source PSO J271.8353+46.9326 found in the database.
Source PSO J277.3873+26.0116 found in the database.
Source PSO J299.5032+55.0722 found in the database.
Source PSO J323.4300-07.5353 found in the database.
Source PSO J345.2949+07.7665 found in the database.
Source PSO J348.5125+69.6598 found in the database.
[['PSO J000.2794+16.6237'], ['PSO J009.8334+58.5781'], ['WISEA J032301.88+562558.0'], ['PSO J069.4987+19.6346'

## Generate Seperate CSV Files with Fundamental Parameters Organized by Model

In [2]:
import pandas as pd 

df = pd.read_csv("sanghi23/Ultracool_Fundamental_Properties_Table.csv")


atmo_cols = ["name_simbadable", "ra_j2000_formula", "dec_j2000_formula", "teff_atmo", "radius_atmo", "logg_atmo", "log_lbol_lsun", "log_lbol_lsun_err"]
evo_cols = ["name_simbadable", "ra_j2000_formula", "dec_j2000_formula",   "teff_evo", "teff_evo_err", "radius_evo", "radius_evo_err", "mass_evo", "mass_evo_err", "logg_evo", "logg_evo_err"]

atmo_df = df[atmo_cols]
evo_df = df[evo_cols]

atmo_df.to_csv('FundamentalPropertiesbyAtmoModel.csv', index=False)
evo_df.to_csv('FundamentalPropertiesbyEvoModel.csv', index=False)

In [10]:
## Ingest New Fundamental Parameters
evo_cols = ["name", "ra_j2000_formula", "dec_j2000_formula", "teff_evo", "teff_evo_err", "radius_evo", "radius_evo_err", "mass_evo", "mass_evo_err", "logg_evo", "logg_evo_err"]
evo_table = pd.read_csv('FundamentalPropertiesbyEvoModel.csv')

modeled_parameters_evo_ingest_dict = [
    {'source': row['name'],
     'ra': row['ra_j2000_formula'], 
     'dec': row['dec_j2000_formula'],
     'Radius': {'value': row['radius_evo'], 'value_error': row['radius_evo_err'], 'parameter': "radius", 'unit': 'R_jup', "model": "evolutionary"},
     'log_g': {'value': row['logg_evo'], 'value_error': row['logg_evo_err'], 'parameter': "log g", 'unit': 'dex', "model": "evolutionary"},
     'T_eff': {'value': row['teff_evo'], 'value_error': row['teff_evo_err'], 'parameter': "T eff", 'unit': 'K', "model": "evolutionary"},
     'Mass': {'value': row['mass_evo'], 'value_error': row['mass_evo_err'], 'parameter': "mass", 'unit': 'M_jup', "model": "evolutionary"},
     'reference': "Sang23"
     } for _, row in evo_table.iterrows()
]

modeled_parameters_evo_ingest_dict
found = []
for i, source_dict in enumerate(modeled_parameters_evo_ingest_dict):  
    found_source = find_source_in_db(db, source_dict['source'], ra=source_dict['ra'], dec=source_dict['dec'])
    found.append(found_source)
print(found)
    #source_dict['source'] = found_source[0]


01/03/2025 05:25:31PM INFO: SDSS J000013.54+255418.6: No Simbad match, trying coord search around 0.0565, 25.905 deg
01/03/2025 05:25:31PM INFO: WISEA J000131.93-084126.9: No Simbad match, trying coord search around 0.382, -8.69 deg
01/03/2025 05:25:31PM INFO: 2MASS J00054844-2157196: No Simbad match, trying coord search around 1.4519, -21.9555 deg
01/03/2025 05:25:31PM INFO: 2MASSI J0006205-172051: No Simbad match, trying coord search around 1.5854, -17.3475 deg
01/03/2025 05:25:31PM INFO: WISEA J000622.67-131955.2: No Simbad match, trying coord search around 1.5953, -13.3308 deg
01/03/2025 05:25:31PM INFO: SDSS J000632.60+140606.3: No Simbad match, trying coord search around 1.6358, 14.1018 deg
01/03/2025 05:25:31PM INFO: PSO J002.0878+52.0687: No Simbad match, trying coord search around 2.0877, 52.0697 deg
01/03/2025 05:25:31PM INFO: 2MASS J00100009-2031122: No Simbad match, trying coord search around 2.5004, -20.5201 deg
01/03/2025 05:25:31PM INFO: 2MASSI J0013578-223520: No Simbad

In [11]:
found

[[],
 ['PSO J000.2794+16.6237'],
 [],
 ['SDSS J000250.98+245413.8'],
 [],
 [],
 [],
 [],
 ['2MASS J00070787-2458042'],
 [],
 [],
 ['2MASS J00132229-1143006'],
 [],
 [],
 [],
 [],
 [],
 ['SDSS J001608.44-004302.3'],
 [],
 [],
 [],
 [],
 ['PSO J004.7148+51.8918'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['2MASSI J0028394+150141'],
 ['2MASS J00285545-1927165'],
 [],
 ['2MASS J00303013-1450333'],
 [],
 [],
 [],
 [],
 [],
 ['2MASS J00325937+1410371'],
 [],
 [],
 ['2MASS J00345684-0706013'],
 [],
 ['SDSS J003843.99+134339.5'],
 ['54 Psc B'],
 ['PSO J009.8334+58.5781'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['WISE J004945.61+215120.0'],
 ['2MASS J00501994-3322402'],
 ['SIPS J0050-1538'],
 [],
 [],
 ['RG 0050-2722'],
 ['2MASS J00531899-3631102'],
 [],
 [],
 [],
 [],
 ['SIPS J0058-0651'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['HIP 6217C'],
 [],
 [],
 [],
 [],
 [],
 ['2MASS J01262109+1428057'],
 ['CTI 012657.5+280202'],
 ['SDSS J012743.51+135420.9'],
 [],
 [],
 [],
 ['PSO 